## Importing Packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import missingno as msno
from numpy import isnan



from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.impute import SimpleImputer

# This is for regression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor 
from sklearn.svm import SVR


# This is for classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.metrics import r2_score

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('importing completed')

importing completed


## Loading Dataset

In [ ]:
data=pd.read_csv('/content/Social Development Bank Loans For 2019.csv')
data.head()

,ID,bank branch,funding type,funding classification,customer sector,financing value,installment value,cashing date,sex,age,social status,special needs,number of family members,saving loan,income
0,1.0,Tabūk,social,family,government employee,60000.0,>= 1000,2019/02,MALE,>= 30,married,No,>= 05,No,< 5000
1,2.0,Hail,project,solution,NaN,160000.0,>= 1000,2019/01,MALE,< 30,single,No,< 02,No,< 5000
2,3.0,Tabūk,social,marriage,government employee,60000.0,>= 1000,2019/02,MALE,< 30,married,No,>= 02,No,>= 7500
3,4.0,Medina,social,marriage,employee of a government company,60000.0,< 1000,2019/03,MALE,< 30,married,No,>= 10,No,>= 5000
4,5.0,Medina,social,family,private sector employee,60000.0,>= 1000,2019/02,FEMALE,>= 30,divorced,No,>= 02,No,>= 10000


### knowing the data

In [ ]:
print('Shape of dataset is ', data.shape)

Shape of dataset is  (11175, 15)


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11175 entries, 0 to 11174
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        11175 non-null  float64
 1   bank branch               11175 non-null  object 
 2   funding type              11175 non-null  object 
 3   funding classification    11175 non-null  object 
 4   customer sector           7225 non-null   object 
 5   financing value           11175 non-null  float64
 6   installment value         11175 non-null  object 
 7   cashing date              11175 non-null  object 
 8   sex                       11175 non-null  object 
 9   age                       11169 non-null  object 
 10  social status             11175 non-null  object 
 11  special needs             11175 non-null  object 
 12  number of family members  11132 non-null  object 
 13  saving loan               11175 non-null  object 
 14  income

In [ ]:
data.describe()

,ID,financing value
count,11175.000000,1.117500e+04
mean,5588.000000,6.396923e+04
std,3226.088963,4.743736e+04
min,1.000000,1.800000e+04
25%,2794.500000,6.000000e+04
50%,5588.000000,6.000000e+04
75%,8381.500000,6.000000e+04
max,11175.000000,2.116000e+06


In [ ]:
#Checking for the null values in the dataset
data.isnull().sum()

ID                             0
bank branch                    0
funding type                   0
funding classification         0
customer sector             3950
financing value                0
installment value              0
cashing date                   0
sex                            0
age                            6
social status                  0
special needs                  0
number of family members      43
saving loan                    0
income                       114
dtype: int64

**Explanation about missing values in our dataset.**

we have 4 column have missing values.

1- column customer sector has 3950 values is missing.

2- column age has 6 values is missing.

3- column number of family members has 43 values is missing.

4- column income has 114 values is missing.

## Data Preprocessing

### Handling Missing values

In [ ]:
def missing (data):
    missing_number = data.isnull().sum().sort_values(ascending=False)
    missing_percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
    missing_values = pd.concat([missing_number, missing_percent*100], axis=1, keys=['Missing_Number', 'Missing_Percent %'])
    return missing_values

missing(data)

,Missing_Number,Missing_Percent %
customer sector,3950,35.346756
income,114,1.020134
number of family members,43,0.384787
age,6,0.053691
ID,0,0.000000
bank branch,0,0.000000
funding type,0,0.000000
funding classification,0,0.000000
financing value,0,0.000000
installment value,0,0.000000


In [ ]:
# See what is the frequent value in each column (for missing values columns).
# customer sector coulmn
sector_freq = data['customer sector'].mode()[0]
print(sector_freq)

# income column
income_freq = data['income'].mode()[0]
print(income_freq)

# number of family members column
family_num_freq = data['number of family members'].mode()[0]
print(family_num_freq)

# age column
age_freq = data['age'].mode()[0]
print(age_freq)

government employee
>= 7500
>= 02
< 30


In [ ]:
# Fill in the Missing Values using the Simple Imputer with the Most Frequent strategy
imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
imputer = imputer.fit(data[['customer sector', 'income', 'number of family members', 'age']])
data[['customer sector', 'income','number of family members', 'age']] = imputer.transform(
    data[['customer sector', 'income', 'number of family members', 'age']])

**Each column's missing values were replaced with its most frequent value.** 

**We filled in the missing values using the most frequent strategy only because the data type of these columns is Categorical.**

In [ ]:
# Check if missing values is filling in each column 
print('Customer sector missing values after handling = ', data['customer sector'].isnull().sum())
print('Income missing values after handling = ', data['income'].isnull().sum())
print('Number of family members missing values after handling = ', data['number of family members'].isnull().sum())
print('Age missing values after handling = ', data['age'].isnull().sum())

Customer sector missing values after handling =  0
Income missing values after handling =  0
Number of family members missing values after handling =  0
Age missing values after handling =  0


In [ ]:
# Re-check the missing values function.
def missing (data):
    missing_number = data.isnull().sum().sort_values(ascending=False)
    missing_percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
    missing_values = pd.concat([missing_number, missing_percent*100], axis=1, keys=['Missing_Number', 'Missing_Percent %'])
    return missing_values

missing(data)

,Missing_Number,Missing_Percent %
ID,0,0.0
bank branch,0,0.0
funding type,0,0.0
funding classification,0,0.0
customer sector,0,0.0
financing value,0,0.0
installment value,0,0.0
cashing date,0,0.0
sex,0,0.0
age,0,0.0


### Feature Engineering

In [ ]:
# Delete unneeded coulmns ID,cashing date and social status.
data.drop(['ID'], axis=1, inplace=True)
data.drop(['cashing date'], axis=1, inplace=True)
data.drop(['social status'], axis=1, inplace=True)
data.head(5)

,bank branch,funding type,funding classification,customer sector,financing value,installment value,sex,age,special needs,number of family members,saving loan,income
0,Tabūk,social,family,government employee,60000.0,>= 1000,MALE,>= 30,No,>= 05,No,< 5000
1,Hail,project,solution,government employee,160000.0,>= 1000,MALE,< 30,No,< 02,No,< 5000
2,Tabūk,social,marriage,government employee,60000.0,>= 1000,MALE,< 30,No,>= 02,No,>= 7500
3,Medina,social,marriage,employee of a government company,60000.0,< 1000,MALE,< 30,No,>= 10,No,>= 5000
4,Medina,social,family,private sector employee,60000.0,>= 1000,FEMALE,>= 30,No,>= 02,No,>= 10000


In [ ]:
# Count the values in the bank branch column
data['bank branch'].value_counts()

Riyadh             1661
Dammam             1171
Jeddah             1000
Medina              861
Abha                730
Mecca               708
Jazan               523
Khamis Mushait      475
Tabūk               462
Taif                450
Buraydah            386
Najrān              351
Al Bahah            227
Hafar Al Batin      226
Kharj               221
Hail                203
Yanbu               195
Al Qunfudhah        184
Arar                181
Al Jowf             163
Al Qurayyat         140
Bisha               127
Al Majma'ah         124
Ar Rass             121
Duwadimi            109
Wadi ad-Dawasir      96
Al Namas             80
Name: bank branch, dtype: int64

In [ ]:
# Replace the cities in the bank branch columns with their main region.
# Instead of 27 different values, we ended up with 5 regions. 
Central_Region = ['Riyadh', "Kharj", "Al Majma'ah", "Wadi ad-Dawasir", "Duwadimi"]
Eastern_Region = ["Dammam", "Hafar Al Batin"]
Southern_Region = ["Abha", "Khamis Mushait", "Al Bahah", "Jazan", "Najrān", "Bisha"] 
Western_Region = ["Jeddah", "Yanbu", "Mecca", "Medina", "Taif", "Al Qunfudhah"]
Northern_Region = ["Tabūk", "Buraydah", "Hail", "Arar", "Al Jowf", "Ar Rass", "Al Namas", "Al Qurayyat"]

data['bank branch'] = data['bank branch'].replace(Central_Region,'Central Region')
data['bank branch'] = data['bank branch'].replace(Eastern_Region,'Eastern Region')
data['bank branch'] = data['bank branch'].replace(Southern_Region,'Southern Region')
data['bank branch'] = data['bank branch'].replace(Western_Region,'Western Region')
data['bank branch'] = data['bank branch'].replace(Northern_Region,'Northern Region')
data.head()

,bank branch,funding type,funding classification,customer sector,financing value,installment value,sex,age,special needs,number of family members,saving loan,income
0,Northern Region,social,family,government employee,60000.0,>= 1000,MALE,>= 30,No,>= 05,No,< 5000
1,Northern Region,project,solution,government employee,160000.0,>= 1000,MALE,< 30,No,< 02,No,< 5000
2,Northern Region,social,marriage,government employee,60000.0,>= 1000,MALE,< 30,No,>= 02,No,>= 7500
3,Western Region,social,marriage,employee of a government company,60000.0,< 1000,MALE,< 30,No,>= 10,No,>= 5000
4,Western Region,social,family,private sector employee,60000.0,>= 1000,FEMALE,>= 30,No,>= 02,No,>= 10000


In [ ]:
# Count the values in the bank branch column after we replace them.
data['bank branch'].value_counts()

Western Region     3398
Southern Region    2433
Central Region     2211
Northern Region    1736
Eastern Region     1397
Name: bank branch, dtype: int64

#### Replace certain columns values with 0 and 1



In [ ]:
# Replace (Yes/No) columns with 0/1 (special needs and saving loan features)
data = data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
convert = {'Yes': 0, 'No': 1, np.nan: 0}
data = data.replace({'special needs': convert, 'saving loan'	: convert})

# Replace sex column with 0 -> male , 1 -> female
data = data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
convert = {'MALE': 0, 'FEMALE': 1, np.nan: 0}
data = data.replace({'sex': convert})

In [ ]:
data.head(5)

,bank branch,funding type,funding classification,customer sector,financing value,installment value,sex,age,special needs,number of family members,saving loan,income
0,Northern Region,social,family,government employee,60000.0,>= 1000,0,>= 30,1,>= 05,1,< 5000
1,Northern Region,project,solution,government employee,160000.0,>= 1000,0,< 30,1,< 02,1,< 5000
2,Northern Region,social,marriage,government employee,60000.0,>= 1000,0,< 30,1,>= 02,1,>= 7500
3,Western Region,social,marriage,employee of a government company,60000.0,< 1000,0,< 30,1,>= 10,1,>= 5000
4,Western Region,social,family,private sector employee,60000.0,>= 1000,1,>= 30,1,>= 02,1,>= 10000


#### Label Encoding

In [ ]:
# Apply Label Encoding to convert certain columns from a categorical type into a numerical one.
# Create a list of the columns to be converted into numerical values.
cols = ['installment value', 'age', 'number of family members', 'income']

# Encode labels of multiple columns at once
data[cols] = data[cols].apply(LabelEncoder().fit_transform)

# Print head
data.head()

,bank branch,funding type,funding classification,customer sector,financing value,installment value,sex,age,special needs,number of family members,saving loan,income
0,Northern Region,social,family,government employee,60000.0,1,0,1,1,2,1,0
1,Northern Region,project,solution,government employee,160000.0,1,0,0,1,0,1,0
2,Northern Region,social,marriage,government employee,60000.0,1,0,0,1,1,1,3
3,Western Region,social,marriage,employee of a government company,60000.0,0,0,0,1,3,1,2
4,Western Region,social,family,private sector employee,60000.0,1,1,1,1,1,1,1


**Columns values before the encoding and after:**
- The unique values of the (installment value) column are ['>= 1000', '< 1000'], and after encoding become [1, 0].

- The unique values of the (age) column are ['>= 30', '< 30', '>= 60', '>= 40'], and after encoding become [1, 0, 3, 2].

- The unique values of the (number of family member) column are ['>= 05', '< 02', '>= 02', '>= 10'], and after encoding become [2, 0, 1, 3].

- The unique values of the (income) column are ['< 5000', '>= 7500', '>= 5000', '>= 10000'], and after encoding become [0, 3, 2, 1].

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11175 entries, 0 to 11174
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   bank branch               11175 non-null  object 
 1   funding type              11175 non-null  object 
 2   funding classification    11175 non-null  object 
 3   customer sector           11175 non-null  object 
 4   financing value           11175 non-null  float64
 5   installment value         11175 non-null  int64  
 6   sex                       11175 non-null  int64  
 7   age                       11175 non-null  int64  
 8   special needs             11175 non-null  int64  
 9   number of family members  11175 non-null  int64  
 10  saving loan               11175 non-null  int64  
 11  income                    11175 non-null  int64  
dtypes: float64(1), int64(7), object(4)
memory usage: 1.0+ MB
